# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

In [4]:
from langchain.agents import create_agent

In [5]:
agent = create_agent(
    model="openai:gpt-5-mini",
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [6]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Why do programmers prefer dark mode?

Because light attracts bugs.

Want another one?


## values
You have seen this streaming mode in our examples so far. 

In [7]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Why do programmers prefer dark mode?

Because light attracts bugs.

Want another one?


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [8]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

Sunbeam Sunday

Sun tiptoes in with a giggle and grin,
waking the house where the day will begin.
Pancakes parade in a syrupy swirl,
and laughter pops up from every boy and girl.

A backyard orchestra — whistles and wings,
a choir of robins and some kettle-bell rings.
Blankets become boats, cushions the sea,
and every small captain's the captain to be.

We plant tiny wishes in a jar by the door,
reading them later when we dream and explore.
Handprints and stories, a fort made of light,
sharing our secrets till hush falls the night.

Stars tuck us in with a twinkle and wink,
and morning comes softly — more wonders to think.

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [11]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='b4f050c0-07b4-425f-a047-f7ab1b2eae6b')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='b4f050c0-07b4-425f-a047-f7ab1b2eae6b'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 132, 'total_tokens': 220, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-COTjjFeNU1nQV6U9Us6ERasaw3iB4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--1f64e741-f46d-4aa1-bf8b-51baec421a72-0', tool_calls=[{'name': 'get_we

In [12]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: San Francisco')
('custom', 'Acquired data for city: San Francisco')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])